# Python learning to rank (LTR) toolkit についての調査

ランク学習の１実装事例である「Python learning to rank (LTR) toolkit」について調査しました。

こちらを参考にいたしました。

https://github.com/jma127/pyltr

https://github.com/jma127/pyltr/blob/master/README.rst

## (1) 概要

Python 言語により実装されております。

実装されているのは「LambdaMART」と呼ばれる、決定木アルゴリズムの一種を使用したランク学習モデル、とのことです。


- LambdaMARTアルゴリズムについての解説は以下URLに記載されています
 
　　https://wellecks.wordpress.com/tag/lambdamart/

　　https://www.microsoft.com/en-us/research/publication/from-ranknet-to-lambdarank-to-lambdamart-an-overview/
 
  
- MART（勾配ブースティング決定木）についての参考文献は以下のURLにあります
  
　　http://smrmkt.hatenablog.jp/entry/2015/04/28/210039 
 
 
- 「LambdaMART」は、手法としては pairwise/listwise に分類されるようです。

　　こちら (https://en.wikipedia.org/wiki/Learning_to_rank#Pointwise_approach) の「List of methods」をご参照
 

- 入力データ（教師データ）形式は<a href="01-SVMRank-outline.ipynb"><b>SVMRank</b></a>と同じものが使用できます。

### (1-1) 教師データ

クエリーごとに、素性（featureベクトルと等価）と、そのランクを教師データとして用意します。

ランクの値が大きいほど、そのクエリーIDにおけるランキングが高い素性である、という意味になるようです。

- 教師データのフォーマット：(fit関数の引数。feature=10件とします)

| ランク | クエリーID | feature1 |  feature2 | ・・・ |  feature10 | 
| :---: | :---: | :---: |  :---: | :---: |  :---: | 
| 3 | qid:1 | 1:0.5 |  2:0.0 | ・・・ |  10:0.5 | 
| 2 | qid:1 | 1:0.2 |  2:1.0 | ・・・ |  10:0.1 | 
| 1 | qid:1 | 1:0.0 |  2:1.0 | ・・・ |  10:0.0 | 
| 3 | qid:2 | 1:0.0 |  2:0.1 | ・・・ |  10:0.2 | 
| 2 | qid:2 | 1:1.0 |  2:0.3 | ・・・ |  10:0.2 | 
| 1 | qid:2 | 1:1.0 |  2:0.5 | ・・・ |  10:1.0 | 


### (1-2) 学習

教師データを学習処理（LambdaMART）に渡すと、fit関数により学習を行い、モデルが生成されます。

### (1-3) 予測


テストデータをpredict関数に渡すと、予測結果が生成され、テストデータについてのランキング・スコアが得られます。


- テストデータのフォーマット：(predict関数の引数＝前述の教師データのフォーマットと同一)

| ランク | クエリーID | feature1 |  feature2 | ・・・ |  feature10 | 
| :---: | :---: | :---: |  :---: | :---: |  :---: | 
| 1 | qid:9 | 1:0.0 |  2:0.0 | ・・・ |  10:0.0 | 
| 1 | qid:9 | 1:0.1 |  2:1.0 | ・・・ |  10:0.1 | 
| 1 | qid:9 | 1:0.5 |  2:0.0 | ・・・ |  10:0.3 | 

　　（テストデータの１番目の列の値は、predict処理には影響しないようです。【後述】）


- 予測結果データのフォーマット：(predict関数の戻り＝テストデータと並びが同じになるようです)

| ランキング・スコア |
| :---: |
| -0.5 |
| 0.5 |
| 1.5 |

ただしこのランキング・スコアは、純粋にテストデータとランキング結果の対応だけに使用される想定であり、その値自体が何らかの指標となるものではないようです。

（すなわち、テストデータをランキング・スコアの降順に並べ替えて、ランキングを得る・・・といった利用を想定している様子）

したがって、テストデータが１件しかないばあい、この手法の予測結果は意味を持たないかと存じます。

## (2) 環境準備

### (2-1) GitHub からファイルを取得

```
MacBookPro-makmorit-jp:GitHub makmorit$ git clone https://github.com/jma127/pyltr.git
Cloning into 'pyltr'...
remote: Counting objects: 281, done.
remote: Total 281 (delta 0), reused 0 (delta 0), pack-reused 281
Receiving objects: 100% (281/281), 50.50 KiB | 0 bytes/s, done.
Resolving deltas: 100% (164/164), done.
MacBookPro-makmorit-jp:GitHub makmorit$ ls -al pyltr
total 56
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 .
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 ..
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 .git
-rw-r--r--   1 makmorit  staff   259 May  3 13:55 .gitignore
-rw-r--r--   1 makmorit  staff   182 May  3 13:55 .travis.yml
-rw-r--r--   1 makmorit  staff  1479 May  3 13:55 LICENSE.txt
-rw-r--r--   1 makmorit  staff  2704 May  3 13:55 README.rst
-rw-r--r--   1 makmorit  staff   211 May  3 13:55 TODO.txt
drwxr-xr-x  11 makmorit  staff   374 May  3 13:55 docs
drwxr-xr-x   7 makmorit  staff   238 May  3 13:55 pyltr
-rwxr-xr-x   1 makmorit  staff    65 May  3 13:55 run_tests.sh
-rw-r--r--   1 makmorit  staff   494 May  3 13:55 setup.py
MacBookPro-makmorit-jp:GitHub makmorit$
```

### (2-2) ユニットテスト実行（環境のベリファイ）

run_tests.sh を実行させます。

事前に、内部でimportされる overrides モジュールを追加導入しています。

```
MacBookPro-makmorit-jp:GitHub makmorit$ cd pyltr
MacBookPro-makmorit-jp:pyltr makmorit$ pwd
/Users/makmorit/GitHub/pyltr
MacBookPro-makmorit-jp:pyltr makmorit$ ls -al
total 56
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 .
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 ..
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 .git
-rw-r--r--   1 makmorit  staff   259 May  3 13:55 .gitignore
-rw-r--r--   1 makmorit  staff   182 May  3 13:55 .travis.yml
-rw-r--r--   1 makmorit  staff  1479 May  3 13:55 LICENSE.txt
-rw-r--r--   1 makmorit  staff  2704 May  3 13:55 README.rst
-rw-r--r--   1 makmorit  staff   211 May  3 13:55 TODO.txt
drwxr-xr-x  11 makmorit  staff   374 May  3 13:55 docs
drwxr-xr-x   7 makmorit  staff   238 May  3 13:55 pyltr
-rwxr-xr-x   1 makmorit  staff    65 May  3 13:55 run_tests.sh
-rw-r--r--   1 makmorit  staff   494 May  3 13:55 setup.py
MacBookPro-makmorit-jp:pyltr makmorit$ 
MacBookPro-makmorit-jp:pyltr makmorit$ pip3 install overrides
Collecting overrides
  Downloading overrides-1.7.tar.gz
Building wheels for collected packages: overrides
  Running setup.py bdist_wheel for overrides ... done
  Stored in directory: /Users/makmorit/Library/Caches/pip/wheels/93/7f/68/cb4e994316c6b40d4ccf468475267fac7105459ee7c51fef9f
Successfully built overrides
Installing collected packages: overrides
Successfully installed overrides-1.7
MacBookPro-makmorit-jp:pyltr makmorit$ ./run_tests.sh
pyltr.metrics.tests.test_ap.TestAP.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_ap.TestAP.test_evaluate ... ok
pyltr.metrics.tests.test_dcg.TestDCG.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_dcg.TestDCG.test_evaluate ... ok
pyltr.metrics.tests.test_dcg.TestNDCG.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_dcg.TestNDCG.test_evaluate ... ok
pyltr.metrics.tests.test_err.TestERR.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_err.TestERR.test_evaluate ... ok
pyltr.metrics.tests.test_kendall.TestKendallTau.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_kendall.TestKendallTau.test_evaluate ... ok
pyltr.metrics.tests.test_roc.TestAUCROC.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_roc.TestAUCROC.test_evaluate ... ok

----------------------------------------------------------------------
Ran 12 tests in 0.532s

OK
MacBookPro-makmorit-jp:pyltr makmorit$ 
```

### (2-3) テストデータの準備

<a href="04-sofia-ml-evaluation.ipynb"><b>sofia-ml</b></a> の調査で使用した、Microsoft のランク学習用ベンチマークデータ（MSLR）を、一部抜粋してテストデータを作成しました。

MSLRのデータセットは非常に大きいので、教師データ（train.txt）と正解ラベル付きのテストデータ（vali.txt）を、それぞれ10,000件／2,500件ずつ切り出して使用します。

テストデータは、教師データ（train.txt）の先頭２０件を拾い出し、一番最初の列にユニークなラベルを付しています。

```
MacBookPro-makmorit-jp:pyltr makmorit$ pwd
/Users/makmorit/GitHub/pyltr
MacBookPro-makmorit-jp:pyltr makmorit$ ls -al data
total 28536
drwxr-xr-x   7 makmorit  staff       238 May  6 12:54 .
drwxr-xr-x  15 makmorit  staff       510 May  3 15:58 ..
-rw-r--r--@  1 makmorit  staff      6148 May  6 12:54 .DS_Store
drwxr-xr-x   5 makmorit  staff       170 May  6 11:47 backup
-rwxr-----@  1 makmorit  staff     23525 May  6 12:14 test.txt
-rwxr-----@  1 makmorit  staff  11621820 May  6 11:53 train.txt
-rwxr-----@  1 makmorit  staff   2952380 May  6 11:55 vali.txt
MacBookPro-makmorit-jp:pyltr makmorit$ head -20 data/train.txt
3 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 （以下略）
3 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:406 12:0 13:5 14:5 15:416 （以下略）
1 qid:1 1:3 2:0 3:2 4:0 5:3 6:1 7:0 8:0.666667 9:0 10:1 11:146 12:0 13:3 14:7 15:156 （以下略）
3 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:287 12:1 13:4 14:7 15:299 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:2009 12:2 13:4 14:7 15:2022 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:935 12:3 13:4 14:7 15:949 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1363 12:4 13:4 14:7 15:1378 （以下略）
3 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:489 12:0 13:4 14:10 15:503 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1295 12:2 13:4 14:7 15:1308 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:510 12:0 13:4 14:5 15:519 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:164 12:1 13:10 14:10 15:185 （以下略）
1 qid:1 1:3 2:1 3:3 4:0 5:3 6:1 7:0.333333 8:1 9:0 10:1 11:666 12:3 13:4 14:7 15:680 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1475 12:1 13:4 14:7 15:1487 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1601 12:0 13:1593 14:5 15:3199 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1268 12:0 13:4 14:7 15:1279 （以下略）
1 qid:1 1:3 2:0 3:2 4:2 5:3 6:1 7:0 8:0.666667 9:0.666667 10:1 11:741 12:0 13:9 14:12 15:762 （以下略）
1 qid:1 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10:0 11:21 12:3 13:3 14:10 15:37 （以下略）
3 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:327 12:1 13:3 14:6 15:337 （以下略）
1 qid:1 1:3 2:0 3:2 4:2 5:3 6:1 7:0 8:0.666667 9:0.666667 10:1 11:741 12:0 13:9 14:13 15:763 （以下略）
1 qid:1 1:3 2:0 3:0 4:0 5:3 6:1 7:0 8:0 9:0 10:1 11:2032 12:0 13:6 14:13 15:2051 （以下略）
MacBookPro-makmorit-jp:pyltr makmorit$ head -10 data/vali.txt
1 qid:10 1:2 2:0 3:0 4:0 5:2 6:0.666667 7:0 8:0 9:0 10:0.666667 11:835 12:0 13:8 14:10 15:853  （以下略）
1 qid:10 1:1 2:0 3:1 4:3 5:3 6:0.333333 7:0 8:0.333333 9:1 10:1 11:10 12:0 13:9 14:11 15:30  （以下略）
2 qid:10 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:557 12:0 13:7 14:11 15:575  （以下略）
1 qid:10 1:3 2:0 3:2 4:0 5:3 6:1 7:0 8:0.666667 9:0 10:1 11:522 12:0 13:6 14:8 15:536  （以下略）
2 qid:10 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:59 12:0 13:5 14:5 15:69  （以下略）
3 qid:10 1:3 2:0 3:3 4:1 5:3 6:1 7:0 8:1 9:0.333333 10:1 11:203 12:0 13:7 14:5 15:215  （以下略）
2 qid:10 1:3 2:0 3:3 4:1 5:3 6:1 7:0 8:1 9:0.333333 10:1 11:321 12:0 13:5 14:5 15:331  （以下略）
2 qid:10 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:195 12:0 13:6 14:4 15:205  （以下略）
1 qid:10 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10:0 11:10 12:0 13:2 14:7 15:19  （以下略）
1 qid:10 1:2 2:0 3:1 4:0 5:2 6:0.666667 7:0 8:0.333333 9:0 10:0.666667 11:919 12:0 13:6 14:4 15:929  （以下略）
MacBookPro-makmorit-jp:pyltr makmorit$ cat data/test.txt
1 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:406 12:0 13:5 14:5 15:416 （以下略）
3 qid:1 1:3 2:0 3:2 4:0 5:3 6:1 7:0 8:0.666667 9:0 10:1 11:146 12:0 13:3 14:7 15:156 （以下略）
4 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:287 12:1 13:4 14:7 15:299  （以下略）
5 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:2009 12:2 13:4 14:7 15:2022  （以下略）
6 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:935 12:3 13:4 14:7 15:949  （以下略）
7 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1363 12:4 13:4 14:7 15:1378 （以下略）
8 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:489 12:0 13:4 14:10 15:503  （以下略）
9 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1295 12:2 13:4 14:7 15:1308 （以下略）
10 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:510 12:0 13:4 14:5 15:519  （以下略）
11 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:164 12:1 13:10 14:10 15:185  （以下略）
12 qid:1 1:3 2:1 3:3 4:0 5:3 6:1 7:0.333333 8:1 9:0 10:1 11:666 12:3 13:4 14:7 15:680  （以下略）
13 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1475 12:1 13:4 14:7 15:1487  （以下略）
14 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1601 12:0 13:1593 14:5 15:3199  （以下略）
15 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1268 12:0 13:4 14:7 15:1279  （以下略）
16 qid:1 1:3 2:0 3:2 4:2 5:3 6:1 7:0 8:0.666667 9:0.666667 10:1 11:741 12:0 13:9 14:12 15:762  （以下略）
17 qid:1 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10:0 11:21 12:3 13:3 14:10 15:37  （以下略）
18 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:327 12:1 13:3 14:6 15:337  （以下略）
19 qid:1 1:3 2:0 3:2 4:2 5:3 6:1 7:0 8:0.666667 9:0.666667 10:1 11:741 12:0 13:9 14:13 15:763  （以下略）
20 qid:1 1:3 2:0 3:0 4:0 5:3 6:1 7:0 8:0 9:0 10:1 11:2032 12:0 13:6 14:13 15:2051  （以下略）
MacBookPro-makmorit-jp:pyltr makmorit$ ```

## (3) example を使用した実行例

以下のサイトの記述を参考にしています。

https://github.com/jma127/pyltr/blob/master/README.rst#example

### (3-0) データの読み込み

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../../../pyltr") #<--- ~/GitHub/pyltr
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
print(learning_dir)

/Users/makmorit/GitHub/pyltr


In [3]:
import pyltr

In [4]:
with open('data/train.txt') as trainfile, \
    open('data/vali.txt') as valifile:
    TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
    VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)

### (3-1) 学習処理

学習セットファイル data/train.txt を引数として学習処理を行うと、model オブジェクトにモデルが生成されます。

In [5]:
metric = pyltr.metrics.NDCG(k=10)
model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=2500,
    learning_rate=0.02,
    max_features=0.5,
    query_subsample=0.5,
    max_leaf_nodes=100,
    min_samples_leaf=64,
    verbose=1,
)
model

In [6]:
monitor = pyltr.models.monitors.ValidationMonitor(VX, Vy, Vqids, metric=metric, stop_after=5000)
model.fit(TX, Ty, Tqids, monitor=monitor)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.3833       0.2103       11.60m      C:      0.4012 B:      0.4012 S:  0
    2       0.4821       0.0394       10.88m      C:      0.4398 B:      0.4398 S:  0
    3       0.4731       0.0174       10.66m      C:      0.4590 B:      0.4590 S:  0
    4       0.5359       0.0277       10.33m      C:      0.4761 B:      0.4761 S:  0
    5       0.5057      -0.0127       10.36m      C:      0.4972 B:      0.4972 S:  0
    6       0.4744       0.0019       10.34m      C:      0.4929 B:      0.4972 S:  1
    7       0.4782      -0.0003       10.47m      C:      0.4903 B:      0.4972 S:  2
    8       0.5435      -0.0001       10.48m      C:      0.4987 B:      0.4987 S:  0
    9       0.5346       0.0007       10.50m      C:      0.4945 B:      0.4987 S:  1
   10       0.5303       0.0005       10.52m      C:      0.5091 B:      0.5091 S:  0
   15       0.5367       0.0034       11.09m      C: 

### (3-2) 予測処理

テストデータファイル 'data/test.txt' を引数として予測処理処理を行うと、Epred に予測結果が格納されます。

ちなみに、例をわかりやすくする為、テストデータの１番目の列（下記Eyに対応）にはユニークなラベルを付しています。

（Eyがpredict関数の引数として不要なことから、予測処理には無関係です）

In [7]:
with open('data/test.txt') as evalfile:
    EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

In [8]:
Epred = model.predict(EX)
Epred

array([-0.47400408, -0.61968845, -1.16388842, -0.64531964, -0.70040169,
       -0.69165497, -0.69165497, -0.50771143, -0.66533367, -0.49547753,
       -0.75647554, -0.68963258, -0.69165497, -1.0101818 , -0.69165497,
       -1.34491462, -1.45191238,  1.65792599, -1.34491462, -1.48566336])

### (3-3) 予測結果を参照

Epred に、ランキング・スコアが出力されます。

（ちなみに、テストデータのレコードと同じ並びになっているようです）

これをユーザープログラムなどで降順に整列し、ランキング結果として利用する想定のようです。

In [9]:
'''
    ランキングスコアで降順ソート
'''
ranking_array = []
for index, _ in enumerate(EX):
    ranking_array.append((Eqids[index], Ey[index], Epred[index]))

sorted_array = sorted(ranking_array, key=lambda x:x[2], reverse=True)

In [10]:
'''
    ランキングを表示します
'''
for qid, label, score in sorted_array:
    print('qid=%s label=%d ranking score=%0.3f' % (qid, label, score))

qid=1 label=18 ranking score=1.658
qid=1 label=1 ranking score=-0.474
qid=1 label=10 ranking score=-0.495
qid=1 label=8 ranking score=-0.508
qid=1 label=2 ranking score=-0.620
qid=1 label=4 ranking score=-0.645
qid=1 label=9 ranking score=-0.665
qid=1 label=12 ranking score=-0.690
qid=1 label=6 ranking score=-0.692
qid=1 label=7 ranking score=-0.692
qid=1 label=13 ranking score=-0.692
qid=1 label=15 ranking score=-0.692
qid=1 label=5 ranking score=-0.700
qid=1 label=11 ranking score=-0.756
qid=1 label=14 ranking score=-1.010
qid=1 label=3 ranking score=-1.164
qid=1 label=16 ranking score=-1.345
qid=1 label=19 ranking score=-1.345
qid=1 label=17 ranking score=-1.452
qid=1 label=20 ranking score=-1.486


### (3-4) モデルの評価

calc_mean_random または calc_mean の両関数を用いて行うとのことです。

学習セット（TX）で予測した結果（Tpred）を用いて、評価を行ってみます。

In [11]:
Tpred = model.predict(TX)
Tpred

array([-0.47400408, -0.61968845, -1.16388842, ..., -0.79744169,
       -1.11094436, -0.84445588])

値が1.0に近ければ高スコアと考えられますが・・・ドキュメント等の説明がない為、詳細は不明です。

In [12]:
print('Random ranking:', metric.calc_mean_random(Tqids, Ty))
print('Our model:', metric.calc_mean(Tqids, Ty, Tpred))

Random ranking: 0.31488525443
Our model: 0.603719105934


ソースコードを見た限りでは、少なくとも Accuracy（正解or不正解）の計測値ではないようですが、１に近いほど高スコアということかと思われます。

In [ ]:
'''
    ソースコードの該当部分を抽出
'''
def _exp2_gain(x): # <---- 下記 _gain_fn(t) の実体
    return math.exp(x * _LOG2) - 1.0

class DCG(Metric):
    @overrides
    def evaluate(self, qid, targets):
        return sum(self._gain_fn(t) * self._get_discount(i)
                   for i, t in enumerate(targets) if i < self.k)

    @classmethod
    def _make_discounts(self, n):
        return np.array([1.0 / np.log2(i + 2.0) for i in range(n)])

    def _get_discount(self, i):
        if i >= self.k:
            return 0.0
        while i >= len(self._discounts):
            self._grow_discounts()
        return self._discounts[i]

    def _grow_discounts(self):
        self._discounts = self._make_discounts(len(self._discounts) * 2)

class Metric(object):
    def evaluate_preds(self, qid, targets, preds):
        """Evaluates the metric on a ranked list of targets.
        """
        return self.evaluate(qid, get_sorted_y(targets, preds))
    
    def calc_mean(self, qids, targets, preds):
        """Calculates the mean of the metric among the provided predictions.
        """
        check_qids(qids)
        query_groups = get_groups(qids)
        return np.mean([self.evaluate_preds(qid, targets[a:b], preds[a:b])
                        for qid, a, b in query_groups])